In [ ]:
import sys
import asyncio
import nbformat

from tqdm import tqdm
from pathlib import Path
from nbconvert.exporters import WebPDFExporter

In [ ]:
if sys.platform.startswith("win"):
    try:
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    except Exception:
        pass

def convert_notebook_to_pdf(ipynb_path, output_dir=None, allow_chromium_download=True):
    ipynb_path = Path(ipynb_path)
    output_dir = Path(output_dir) if output_dir else ipynb_path.parent
    output_dir.mkdir(parents=True, exist_ok=True)

    nb = nbformat.read(ipynb_path, as_version=4)

    exporter = WebPDFExporter(allow_chromium_download=allow_chromium_download)
    exporter.exclude_input_prompt = True
    exporter.exclude_output_prompt = True

    pdf_data, _ = exporter.from_notebook_node(nb)

    out_pdf = output_dir / f"{ipynb_path.stem}.pdf"
    out_pdf.write_bytes(pdf_data)
    return out_pdf

def batch_convert_ipynb_to_pdf(input_dir, output_dir=None, skip_existing=True):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir) if output_dir else input_dir

    notebooks = sorted(input_dir.rglob("*.ipynb"))
    if not notebooks:
        print("No .ipynb files found.")
        return

    successes, fails = [], []

    for nb_path in tqdm(notebooks, desc="Converting notebooks", unit="nb"):
        out_pdf = output_dir / f"{nb_path.stem}.pdf"
        if skip_existing and out_pdf.exists():
            continue

        try:
            pdf_path = convert_notebook_to_pdf(nb_path, output_dir)
            successes.append(pdf_path)
        except Exception as e:
            fails.append((nb_path, str(e)))

    print("\n✅ Generated PDFs:")
    for p in successes:
        print(f" - {p}")

    if fails:
        print("\n⚠️ Failures:")
        for nb, err in fails:
            print(f" - {nb}: {err}")

In [ ]:
input_dir = r"C:\Users\lgdan\Documents\MASTER\ML\notebooks"

output_dir = None  

batch_convert_ipynb_to_pdf(input_dir, output_dir)